In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision import models
import h5py
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score
from tqdm import tqdm  # Import tqdm for the progress bar
import os  # Add this import at the beginning of your script

In [2]:
# Load PlantVillage dataset
root_dir = '/kaggle/input/plantvillage/PlantVillage'  # Update with actual path
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [3]:
train_dataset = ImageFolder(root=os.path.join(root_dir, 'train'), transform=train_transform)
val_dataset = ImageFolder(root=os.path.join(root_dir, 'val'), transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=8)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=8)


/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [4]:
# Load the pretrained model from the .h5 file
class FineTunedModel(nn.Module):
    def __init__(self, num_classes=38):
        super(FineTunedModel, self).__init__()
        # Load pretrained MobileNet
        self.base_model = models.mobilenet_v2(pretrained=False)
        self.base_model.classifier = nn.Linear(self.base_model.last_channel, num_classes)

        # Load weights from .h5
        with h5py.File("/kaggle/input/h5file/MOCO_model_weights_final.h5", 'r') as f:
            for name, param in self.base_model.named_parameters():
                if name in f:
                    param.data.copy_(torch.tensor(np.array(f[name])))

    def forward(self, x):
        return self.base_model(x)

In [5]:
# Initialize the fine-tuned model
num_classes = len(train_dataset.classes)
model = FineTunedModel(num_classes=num_classes).cuda()


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [6]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.01, weight_decay=0.01)


In [7]:
# Fine-tuning loop with progress bar
num_epochs = 30 # Adjust as needed
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    # Progress bar for training
    progress_bar = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{num_epochs}]")

    # Training
    for images, labels in progress_bar:
        images, labels = images.cuda(), labels.cuda()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Update progress bar
        progress_bar.set_postfix(loss=loss.item())

    train_accuracy = 100 * correct / total
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Avg Loss: {avg_loss:.4f}, Training Accuracy: {train_accuracy:.2f}%")

Epoch [1/25]: 100%|██████████| 679/679 [01:59<00:00,  5.70it/s, loss=1.48]


Epoch [1/25], Avg Loss: 2.2803, Training Accuracy: 37.81%


Epoch [2/25]: 100%|██████████| 679/679 [01:57<00:00,  5.79it/s, loss=1.01] 


Epoch [2/25], Avg Loss: 1.2754, Training Accuracy: 61.61%


Epoch [3/25]: 100%|██████████| 679/679 [01:57<00:00,  5.78it/s, loss=0.761]


...

Epoch [23/25], Avg Loss: 0.1625, Training Accuracy: 94.77%


Epoch [24/25]: 100%|██████████| 679/679 [01:57<00:00,  5.78it/s, loss=0.192] 


Epoch [24/25], Avg Loss: 0.1610, Training Accuracy: 94.73%


Epoch [25/25]: 100%|██████████| 679/679 [01:57<00:00,  5.77it/s, loss=0.179] 

Epoch [25/25], Avg Loss: 0.1524, Training Accuracy: 95.04%


In [8]:
# Validation
model.eval()
all_labels = []
all_predictions = []

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.cuda(), labels.cuda()
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)

        # Store labels and predictions for metrics calculation
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

# Calculate metrics
val_accuracy = 100 * (sum(np.array(all_predictions) == np.array(all_labels)) / len(all_labels))
precision = precision_score(all_labels, all_predictions, average='macro')  # Macro for multi-class
recall = recall_score(all_labels, all_predictions, average='macro')        # Macro for multi-class
f1 = f1_score(all_labels, all_predictions, average='macro')                # Macro for multi-class

# Print metrics
print(f"Epoch [{epoch+1}/{num_epochs}], Validation Accuracy: {val_accuracy:.2f}%, "
      f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

Epoch [25/25], Validation Accuracy: 96.92%, Precision: 0.9642, Recall: 0.9599, F1 Score: 0.9611


In [12]:
# Save the fine-tuned model
torch.save(model.state_dict(), "moco_fine_tuned_model.pth")

In [13]:
# Save the fine-tuned model in .h5 format
import h5py

def save_model_to_h5(model, filepath="moco_fine_tuned_model.h5"):
    with h5py.File(filepath, 'w') as f:
        for name, param in model.named_parameters():
            f.create_dataset(name, data=param.cpu().detach().numpy())
            
# Save the fine-tuned model weights to an h5 file
save_model_to_h5(model, filepath="moco_fine_tuned_model.h5")
